# Spark Cheat Sheet

This cheat sheet provides a quick reference to common Apache Spark commands and operations. The idea is to have a handy guide for frequently used commands and concepts.

## Introduction

### What is Spark?

Spark is an engine to process data in a distributed way. It can be used in different languages, including Scala, Python (PySpark), Java, and R.


### Why Spark?

The purpose of Spark is to process datasets that are too large to fit into the memory of a single machine. It does this by distributing the data and computations across a cluster of machines.

## Import Data

In [ ]:
from pyspark.sql import SparkSession
import re
from itertools import combinations
from datetime import datetime, timedelta
import pandas as pd

## Best Practices

1. **Use `reduceByKey` instead of `groupByKey`** when possible - it's more efficient
2. **Cache RDDs** that will be reused multiple times
3. **Use broadcast variables** for small lookup tables
4. **Filter early** in your pipeline to reduce data size
5. **Avoid `collect()`** on large datasets - use `take()` or aggregations instead
6. **Use `getattr()`** for safer field access when dealing with inconsistent data
7. **Normalize and validate** data early in the pipeline
8. **Be careful with joins** - they can be expensive. Consider broadcast joins for small datasets
9. **Use meaningful variable names** and add comments for complex transformations
10. **Test on small samples** before running on full datasets

In [ ]:
# Complete workflow example
# 1. Read data
orders_df = spark.read.csv("orders.csv", header=True, inferSchema=True)
products_df = spark.read.csv("products.csv", header=True, inferSchema=True)

# 2. Clean and transform
orders_cleaned = orders_df.rdd.filter(
    lambda r: r.product_id is not None and r.quantity is not None
).map(
    lambda r: (str(r.product_id), int(r.quantity))
)

# 3. Aggregate
qty_by_product = orders_cleaned.reduceByKey(lambda a, b: a + b)

# 4. Get top N
top_10 = qty_by_product.takeOrdered(10, key=lambda kv: -kv[1])

# 5. Enrich with product names (join)
products_kv = products_df.rdd.map(lambda r: (str(r.product_id), r.product_name))
top_10_rdd = spark.sparkContext.parallelize(top_10)
result = top_10_rdd.join(products_kv).map(
    lambda kv: {"name": kv[1][1], "quantity": kv[1][0]}
).collect()

# 6. Display
pd.DataFrame(result)

## Complete Example: Top Products by Sales

In [ ]:
# Collect results and convert to pandas for display
results = rdd.collect()
df_results = pd.DataFrame(results)

if not df_results.empty:
    df_results = df_results.sort_values(by=["column1", "column2"]).reset_index(drop=True)
    display(df_results)
else:
    print("No results")

## Display Results

### Convert to Pandas DataFrame

In [ ]:
# Calculate average by group, then join back
# Step 1: Calculate average by brand
by_brand = products.map(lambda x: (x["brand"], (x["stock"], 1)))
brand_totals = by_brand.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))
brand_avg = brand_totals.mapValues(lambda s: s[0] / s[1] if s[1] else 0.0)

# Step 2: Join products with their brand average
products_kv = products.map(lambda x: (x["brand"], x))
joined = products_kv.join(brand_avg)  # (brand, (product, avg_stock))

# Step 3: Filter based on comparison with average
high_stock = joined.filter(lambda kv: kv[1][0]["stock"] >= 1.2 * kv[1][1])

### Multi-Step Aggregations

In [ ]:
# Calculate percentage after aggregation
result = aggregated.map(lambda kv: {
    "key": kv[0],
    "total": kv[1]["total"],
    "active": kv[1]["active"],
    "active_percent": f"{round(kv[1]['active'] / kv[1]['total'] * 100, 2)}%" if kv[1]["total"] > 0 else "0.0%"
})

### Calculate Percentages

In [ ]:
# Normalize boolean values
def normalize_bool(val):
    if val is None:
        return None
    s = str(val).strip().lower()
    if s in ("true", "1", "yes", "y"):
        return True
    if s in ("false", "0", "no", "n"):
        return False
    return None

# Validate against allowed set
allowed_values = {"REGULAR", "PREMIUM", "BUDGET"}

def normalize_from_set(val, allowed_set):
    if val is None:
        return None
    s = str(val).strip().upper()
    return s if s in allowed_set else None

# Apply normalization
cleaned = rdd.map(lambda r: {
    "segment": normalize_from_set(r.segment, allowed_values),
    "is_active": normalize_bool(r.is_active)
})

### Normalize and Validate Data

In [ ]:
# Extract state code from address
STATE_REGEX = r",\s*([A-Z]{2})\s+\d{5}"

def extract_state(address):
    if not address:
        return None
    match = re.search(STATE_REGEX, address)
    return match.group(1) if match else None

# Extract zip code
def extract_zip(address):
    if not address:
        return None
    match = re.search(r"(\d{5})$", address)
    return match.group(1) if match else None

# Apply extraction
states = rdd.map(lambda r: extract_state(r.address))

## Common Patterns

### Extract Data with Regex

In [ ]:
# Broadcast small datasets to all workers for efficient lookups
top_products_set = set([1, 2, 3, 4, 5])
broadcast_products = spark.sparkContext.broadcast(top_products_set)

# Use broadcast variable in transformations
filtered = rdd.filter(lambda r: r.product_id in broadcast_products.value)

### Broadcast Variables

In [ ]:
# Cache RDD in memory for reuse (important for iterative operations!)
cached_rdd = rdd.cache()

# Use cache when you'll reuse the RDD multiple times
processed = cleaned.map(lambda x: transform(x)).cache()

# Now you can use processed multiple times without recomputation
result1 = processed.filter(lambda x: x.type == "A").count()
result2 = processed.filter(lambda x: x.type == "B").count()

## Optimization Techniques

### Cache and Persist

In [ ]:
# Get top N elements (ascending by default)
top_5_ascending = rdd.takeOrdered(5)

# Get top N elements in descending order (use negative key)
top_5_descending = rdd.takeOrdered(5, key=lambda x: -x)

# Get top 5 by value in key-value pair
top_5_by_value = key_value_rdd.takeOrdered(5, key=lambda kv: -kv[1])

### TakeOrdered

In [ ]:
# Collect all results to driver (be careful with large datasets!)
results = rdd.collect()

# Count number of elements
count = rdd.count()

# Take first N elements
first_10 = rdd.take(10)

# Check if RDD is empty
is_empty = rdd.isEmpty()

## Actions (Trigger Computation)

### Collect, Count, Take

In [ ]:
# Inner join (only matching keys)
# rdd1: (key, value1), rdd2: (key, value2)
joined = rdd1.join(rdd2)  # Result: (key, (value1, value2))

# Left outer join (all keys from left RDD)
left_joined = rdd1.leftOuterJoin(rdd2)  # Result: (key, (value1, Option[value2]))

# Example: Join orders with customers
orders_kv = orders.map(lambda r: (r.customer_id, r))
customers_kv = customers.map(lambda r: (r.customer_id, r))
joined = orders_kv.join(customers_kv)  # (customer_id, (order, customer))

## Join Operations

In [ ]:
# Find maximum value
max_value = rdd.reduce(lambda a, b: a if a > b else b)

# Find max by specific field
max_discount = rdd.map(
    lambda x: (x["state"], x["discount"])
).reduce(lambda a, b: a if a[1] > b[1] else b)

### Reduce

In [ ]:
# Group values by key (returns iterator)
grouped = key_value_rdd.groupByKey()

# Convert to list for easier manipulation
grouped_list = grouped.mapValues(list)

# Example: Group products by order
order_items = rdd.map(lambda r: (r.order_id, r.product_id)).groupByKey().mapValues(list)

### GroupByKey

In [ ]:
# Sum values by key
summed = key_value_rdd.reduceByKey(lambda a, b: a + b)

# Count occurrences by key
counts = rdd.map(lambda x: (x.category, 1)).reduceByKey(lambda a, b: a + b)

# Aggregate multiple metrics
aggregated = rdd.map(
    lambda x: (x.key, {"count": 1, "sum": x.value})
).reduceByKey(
    lambda a, b: {
        "count": a["count"] + b["count"],
        "sum": a["sum"] + b["sum"]
    }
)

## Aggregation Operations

### ReduceByKey

In [ ]:
# FlatMap to generate multiple elements per input row
# Useful for generating pairs from combinations
def generate_pairs(items):
    pairs = []
    for prod1, prod2 in combinations(items, 2):
        if prod1 < prod2:
            pairs.append(((prod1, prod2), 1))
        else:
            pairs.append(((prod2, prod1), 1))
    return pairs

# Apply flatMap to flatten the results
flattened = grouped_rdd.flatMap(lambda kv: generate_pairs(kv[1]))

### FlatMap

In [ ]:
# Map to transform each row into a dictionary
mapped = rdd.map(lambda r: {
    "id": r.id,
    "name": r.name,
    "value": float(r.value)
})

# Map to extract a single field
values = rdd.map(lambda r: r.column_name)

# Map to key-value pairs for grouping
key_value = rdd.map(lambda r: (r.key, r.value))

### Map

In [ ]:
# Filter rows based on a condition
filtered = rdd.filter(lambda r: r.column_name is not None)

# Filter with multiple conditions
filtered = rdd.filter(
    lambda r: r.status == "REFUNDED" and r.amount is not None
)

# Filter with getattr for safer access
filtered = rdd.filter(
    lambda r: getattr(r, "field", None) is not None
)

## Basic RDD Transformations

### Filter

In [ ]:
# Read CSV file with header and infer schema
df = spark.read.csv("path/to/file.csv", header=True, inferSchema=True)

# Convert DataFrame to RDD for transformations
rdd = df.rdd

## Reading Data

### Read CSV to DataFrame

In [ ]:
# Create a Spark session
spark = SparkSession.builder.appName("MyApp").getOrCreate()

# Set log level to reduce noise
spark.sparkContext.setLogLevel("ERROR")

## Setup SparkSession